# MNSYNLPM - Match Exporter and Analyzer for League of Legends

In [2]:
import json
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path

# Charger tous les fichiers JSON du dossier data
data_dir = Path("data")
all_matches = []

for json_file in data_dir.glob("*.json"):
    with open(json_file, "r", encoding="utf-8") as f:
        match_data = json.load(f)
        all_matches.append(match_data)

print(f"Nombre de matchs chargés : {len(all_matches)}")

Nombre de matchs chargés : 9


In [ ]:
from src.constant import VIRGULE

# Extraire TOUTES les statistiques des joueurs Virgule avec calcul par minute
virgule_players_data = []

for match in all_matches:
    game_duration_sec = match.get("game", {}).get("gameDuration", 0)
    game_duration_min = game_duration_sec / 60 if game_duration_sec > 0 else 1
    
    # Pour chaque joueur Virgule, extraire ses stats complètes
    for player_name in VIRGULE:
        if player_name in match:
            player_data = match[player_name]
            champion_data = player_data.get("Champion", {})
            damage_data = player_data.get("Damage", {})
            kda_data = player_data.get("KDA", {})
            first_data = player_data.get("First", {})
            spell_data = player_data.get("Spell", {})
            ping_data = player_data.get("Ping", {})
            
            # Calculer les totaux de pings
            total_pings = sum([
                ping_data.get("onMyWay", 0) or 0,
                ping_data.get("danger", 0) or 0,
                ping_data.get("getBack", 0) or 0,
                ping_data.get("enemyMissing", 0) or 0,
                ping_data.get("assistMe", 0) or 0,
                ping_data.get("retreat", 0) or 0,
                ping_data.get("enemyVision", 0) or 0,
                ping_data.get("hold", 0) or 0,
                ping_data.get("needVision", 0) or 0,
                ping_data.get("push", 0) or 0,
                ping_data.get("visionCleared", 0) or 0,
                ping_data.get("allIn", 0) or 0,
                ping_data.get("basic", 0) or 0,
            ])
            
            # Stats KDA
            kills = kda_data.get("kills", 0) or 0
            deaths = kda_data.get("deaths", 0) or 0
            assists = kda_data.get("assists", 0) or 0
            kda_ratio = (kills + assists) / deaths if deaths > 0 else (kills + assists)
            
            # Compilation complète des stats
            player_stats = {
                "Joueur": player_name,
                "Champion": champion_data.get("championName"),
                "Position": player_data.get("Position"),
                "Victoire": match.get("virgule", {}).get("win", False),
                "Durée (min)": round(game_duration_min, 1),
                
                # KDA
                "Kills": kills,
                "Deaths": deaths,
                "Assists": assists,
                "KDA": round(kda_ratio, 2),
                "Kills/min": round(kills / game_duration_min, 2),
                "Deaths/min": round(deaths / game_duration_min, 2),
                "Assists/min": round(assists / game_duration_min, 2),
                "DoubleKills": kda_data.get("doubleKills", 0) or 0,
                "TripleKills": kda_data.get("tripleKills", 0) or 0,
                "QuadraKills": kda_data.get("quadraKills", 0) or 0,
                "PentaKills": kda_data.get("pentaKills", 0) or 0,
                "LargestMultiKill": kda_data.get("largestMultiKill", 0) or 0,
                "LargestKillingSpree": kda_data.get("largestKillingSpree", 0) or 0,
                
                # Champion Stats
                "Level": champion_data.get("championLevel", 0) or 0,
                "Gold": champion_data.get("gold", 0) or 0,
                "Gold/min": round((champion_data.get("gold", 0) or 0) / game_duration_min, 1),
                "CS": champion_data.get("creeps", 0) or 0,
                "CS/min": round((champion_data.get("creeps", 0) or 0) / game_duration_min, 1),
                "Turrets": champion_data.get("turretTakedowns", 0) or 0,
                "Items Achetés": champion_data.get("itemsPurchased", 0) or 0,
                "Temps Mort (s)": champion_data.get("totalTimeSpentDead", 0) or 0,
                "Temps Vivant Max (s)": champion_data.get("longestTimeSpentLiving", 0) or 0,
                
                # Vision
                "Vision Score": champion_data.get("visionScore", 0) or 0,
                "Vision Score/min": round((champion_data.get("visionScore", 0) or 0) / game_duration_min, 1),
                "Wards Placés": champion_data.get("wardsPlaced", 0) or 0,
                "Wards Détruits": champion_data.get("wardsKilled", 0) or 0,
                "Pinks Achetés": champion_data.get("visionWardsPlaced", 0) or 0,
                
                # Dégâts
                "Dégâts Infligés": damage_data.get("totalDamageDealt", 0) or 0,
                "Dégâts/min": round((damage_data.get("totalDamageDealt", 0) or 0) / game_duration_min, 1),
                "Dégâts Physiques": damage_data.get("physicalDamageDealt", 0) or 0,
                "Dégâts Magiques": damage_data.get("magicDamageDealt", 0) or 0,
                "Dégâts Vrais": damage_data.get("trueDamageDealt", 0) or 0,
                "Dégâts Subis": damage_data.get("totalDamageTaken", 0) or 0,
                "Dégâts Subis/min": round((damage_data.get("totalDamageTaken", 0) or 0) / game_duration_min, 1),
                "Dégâts Bâtiments": damage_data.get("damageDealtToBuildings", 0) or 0,
                "Dégâts Epic Monsters": damage_data.get("damageDealtToEpicMonsters", 0) or 0,
                "Largest Crit": damage_data.get("largestCriticalStrike", 0) or 0,
                "Objectives Volés": damage_data.get("objectivesStolen", 0) or 0,
                
                # Heal
                "Total Heal": damage_data.get("totalHeal", 0) or 0,
                "Heal Alliés": damage_data.get("totalHealsOnTeammates", 0) or 0,
                
                # First
                "First Blood": first_data.get("firstBlood", False),
                "First Tower": first_data.get("firstTower", False),
                
                # Pings
                "Total Pings": total_pings,
                "Pings/min": round(total_pings / game_duration_min, 1),
            }
            
            virgule_players_data.append(player_stats)

# Créer un DataFrame
df_virgule = pd.DataFrame(virgule_players_data)

# Afficher les premières lignes
print(f"\nNombre total de performances : {len(df_virgule)}")
print(f"\nNombre de matchs : {len(all_matches)}")
print(f"\nNombre de joueurs Virgule : {len(VIRGULE)}")
print("\nAperçu des données :")
df_virgule.head(10)


Nombre total de performances : 45

Nombre de matchs : 9

Nombre de joueurs Virgule : 7

Aperçu des données :


,Joueur,Champion,Position,Victoire,Durée (min),Kills,Deaths,Assists,KDA,Kills/min,...,Largest Crit,Objectives Volés,Total Heal,Heal Alliés,Temps CC (s),CC/min,First Blood,First Tower,Total Pings,Pings/min
0,matise,Fiora,TOP,False,45.5,6,7,3,1.29,0.13,...,880,0,18798,3999,175,3.8,True,False,8,0.2
1,Fear of women,DrMundo,JUNGLE,False,45.5,3,5,4,1.40,0.07,...,0,1,46796,0,580,12.8,False,False,57,1.3
2,Sabri,Syndra,MIDDLE,False,45.5,2,4,3,1.25,0.04,...,0,0,2568,0,274,6.0,False,False,29,0.6
3,Ersees,Xayah,BOTTOM,False,45.5,6,7,4,1.43,0.13,...,1096,0,11356,0,94,2.1,False,True,17,0.4
4,nathboy,Rakan,SUPPORT,False,45.5,1,6,12,2.17,0.02,...,0,0,16400,7797,109,2.4,False,False,4,0.1
5,matise,RekSai,TOP,True,41.1,8,4,4,3.00,0.19,...,0,1,33740,0,118,2.9,False,True,8,0.2
6,Fear of women,MonkeyKing,JUNGLE,True,41.1,5,4,9,3.50,0.12,...,241,2,16193,0,214,5.2,True,False,73,1.8
7,Sabri,Ryze,MIDDLE,True,41.1,5,2,4,4.50,0.12,...,0,0,4983,0,38,0.9,False,False,17,0.4
8,Ersees,Varus,BOTTOM,True,41.1,4,3,10,4.67,0.10,...,0,1,3384,0,392,9.5,True,False,10,0.2
9,nathboy,Alistar,SUPPORT,True,41.1,0,3,13,4.33,0.00,...,0,1,12855,7258,163,4.0,True,False,7,0.2


## Statistiques Globales par Joueur

In [ ]:
# Statistiques globales par joueur
stats_by_player = df_virgule.groupby("Joueur").agg({
    "Victoire": ["sum", "count", lambda x: round(x.mean() * 100, 1)],
    "Kills": "mean",
    "Deaths": "mean",
    "Assists": "mean",
    "KDA": "mean",
    "Kills/min": "mean",
    "Gold/min": "mean",
    "CS/min": "mean",
    "Dégâts/min": "mean",
    "Vision Score/min": "mean",
    "Pings/min": "mean",
    "LargestKillingSpree": "max",
    "Objectives Volés": "sum",
}).round(2)

stats_by_player.columns = ["Victoires", "Matchs Joués", "Winrate %", "Kills Moy", "Deaths Moy", 
                            "Assists Moy", "KDA Moy", "Kills/min", "Gold/min", "CS/min", 
                            "Dégâts/min", "Vision/min", "Pings/min", 
                            "Meilleur Spree", "Total Obj Volés"]

print("=== STATISTIQUES GLOBALES PAR JOUEUR ===\n")
stats_by_player.sort_values("Winrate %", ascending=False)

=== STATISTIQUES GLOBALES PAR JOUEUR ===



,Victoires,Matchs Joués,Winrate %,Kills Moy,Deaths Moy,Assists Moy,KDA Moy,Kills/min,Gold/min,CS/min,Dégâts/min,Vision/min,Pings/min,CC/min,Meilleur Spree,Total Obj Volés
Joueur,,,,,,,,,,,,,,,,
Shao Mao,3,6,50.0,5.33,4.33,10.00,5.45,0.17,399.30,7.45,799.52,0.83,0.38,11.15,10,0
Ersees,4,9,44.4,5.44,4.00,9.56,5.83,0.16,500.43,8.68,950.77,0.46,0.24,5.09,7,1
Fear of women,4,9,44.4,6.78,4.89,8.11,4.93,0.20,404.09,5.97,540.42,1.01,1.51,8.64,11,3
matise,4,9,44.4,5.11,5.33,4.78,3.31,0.15,393.31,7.04,712.87,0.63,0.12,21.81,8,1
nathboy,4,9,44.4,1.33,4.33,14.89,5.44,0.04,281.82,1.00,215.80,3.52,0.19,4.29,2,1
Sabri,1,3,33.3,2.33,3.33,3.00,2.08,0.05,338.00,7.53,350.63,0.63,0.33,2.30,2,0


In [ ]:
# Classement par statistiques individuelles
print("=== TOP PERFORMERS ===\n")

categories = {
    "🏆 Meilleur KDA": "KDA",
    "⚔️ Plus de Kills/min": "Kills/min",
    "💀 Moins de Deaths/min": "Deaths/min",
    "💰 Plus de Gold/min": "Gold/min",
    "🌾 Meilleur CS/min": "CS/min",
    "💥 Plus de Dégâts/min": "Dégâts/min",
    "👁️ Meilleur Vision Score/min": "Vision Score/min",
}

for label, col in categories.items():
    if col == "Deaths/min":
        best = df_virgule.nsmallest(1, col)
    else:
        best = df_virgule.nlargest(1, col)
    
    print(f"{label}: {best['Joueur'].values[0]} ({best['Champion'].values[0]}) - {best[col].values[0]}")

=== TOP PERFORMERS ===

🏆 Meilleur KDA: Ersees (Aphelios) - 21.0
⚔️ Plus de Kills/min: matise (Sett) - 0.43
💀 Moins de Deaths/min: matise (KSante) - 0.0
💰 Plus de Gold/min: Ersees (Yunara) - 576.5
🌾 Meilleur CS/min: Ersees (Yunara) - 9.9
💥 Plus de Dégâts/min: Ersees (Aphelios) - 1553.1
👁️ Meilleur Vision Score/min: nathboy (Rakan) - 4.4
🔒 Plus de CC/min: matise (KSante) - 72.5


## Visualisations des Performances

In [8]:
# Graphique : Comparaison des KDA moyens par joueur
avg_kda = df_virgule.groupby("Joueur")["KDA"].mean().sort_values(ascending=False)

fig = px.bar(
    x=avg_kda.index,
    y=avg_kda.values,
    title="KDA Moyen par Joueur",
    labels={"x": "Joueur", "y": "KDA Moyen"},
    color=avg_kda.values,
    color_continuous_scale="Viridis"
)
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
# Radar chart comparant les stats normalisées par minute
stats_normalized = df_virgule.groupby("Joueur").agg({
    "Kills/min": "mean",
    "Assists/min": "mean",
    "Gold/min": "mean",
    "CS/min": "mean",
    "Dégâts/min": "mean",
    "Vision Score/min": "mean",
    "CC/min": "mean",
}).round(2)

fig = go.Figure()

for player in stats_normalized.index:
    fig.add_trace(go.Scatterpolar(
        r=stats_normalized.loc[player].values,
        theta=stats_normalized.columns,
        fill='toself',
        name=player
    ))

fig.update_layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, stats_normalized.max().max()]
        )),
    showlegend=True,
    title="Comparaison des Performances (par minute)"
)

fig.show()

In [ ]:
# Distribution des dégâts par minute
fig = px.box(
    df_virgule,
    x="Joueur",
    y="Dégâts/min",
    color="Joueur",
    title="Distribution des Dégâts par Minute par Joueur",
    points="all"
)

fig.update_layout(
    xaxis_title="Joueur",
    yaxis_title="Dégâts par Minute",
    showlegend=False
)

fig.show()

## Analyse par Position et Champion

In [ ]:
# Performance par position
print("=== STATISTIQUES PAR POSITION ===\n")

stats_by_position = df_virgule.groupby("Position").agg({
    "KDA": "mean",
    "Kills/min": "mean",
    "Gold/min": "mean",
    "CS/min": "mean",
    "Dégâts/min": "mean",
    "Vision Score/min": "mean",
    "Victoire": lambda x: round(x.mean() * 100, 1)
}).round(2)

stats_by_position.columns = ["KDA Moy", "Kills/min", "Gold/min", "CS/min", "Dégâts/min", "Vision/min", "Winrate %"]
stats_by_position.sort_values("KDA Moy", ascending=False)

Statistiques par champion (Champions joués par Virgule) :


kills     deaths     assists         gold     cs level
               mean sum   mean sum    mean sum     mean   mean  mean
champion_name                                                       
Ahri            5.0  10    2.0   4     4.5   9  11646.5  242.0  15.5
Alistar         0.0   0    3.0   3    13.0  13  11008.0   32.0  16.0
Aphelios        8.0   8    1.0   1    13.0  13  16699.0  259.0  16.0
Bard            0.0   0    5.0   5     6.0   6   7511.0   32.0  13.0
Braum           1.0   1    4.0   4    21.0  21   7928.0   26.0  12.0
DrMundo         3.0   3    5.0   5     4.0   4  15237.0  253.0  18.0
Ezreal          3.0   3    4.0   4     2.0   2  13455.0  285.0  15.0
Fiora           6.0   6    7.0   7     3.0   3  18177.0  334.0  20.0
Jax             1.0   1    7.0   7     1.0   1  10030.0  219.0  16.0
KSante          8.0   8    0.0   0     6.0   6  11135.0  185.0  16.0
Lux             4.0   4    7.0   7    12.0  12  13578.0  280.0  17.0
Milio           1.0   1    2.0   2    20.0  20   9194.0   35.0  14.0
MonkeyKing      5.0   5    4.0   4     9.0   9  16061.0  265.0  18.0
Poppy           4.0   4    6.0   6    15.0  15  10947.0   43.0  15.0
Rakan           1.0   1    6.0   6    12.0  12  12546.0   40.0  15.0
RekSai          8.0   8    4.0   4     4.0   4  18329.0  309.0  20.0
Ryze            5.0   5    2.0   2     4.0   4  15954.0  307.0  18.0
Sett           13.0  13    3.0   3     3.0   3  14372.0  201.0  17.0
Swain           3.0   3   10.0  10     8.0   8  12807.0  255.0  19.0
Syndra          2.0   2    4.0   4     3.0   3  14906.0  330.0  18.0
Varus           4.5   9    5.5  11    10.5  21  18699.5  315.0  17.5
Vi             10.0  10   10.0  10     8.0   8  15925.0  202.0  17.0
Viego           1.0   1    6.0   6     3.0   3  11324.0  179.0  15.0
Viktor          3.0   3    3.0   3    13.0  13  11787.0  225.0  17.0
Volibear        7.0   7    4.0   4     9.0   9  12498.0  175.0  16.0
Xayah           6.0   6    7.0   7     4.0   4  22571.0  416.0  18.0
Yunara          6.0   6    4.0   4    16.0  16  12217.0  165.0  13.0
Zaahen          6.0   6    1.0   1    14.0  14  11880.0  173.0  15.0

## Champions les Plus Joués et Performances

In [ ]:
# Champions les plus joués
print("=== TOP CHAMPIONS JOUÉS ===\n")

champ_stats = df_virgule.groupby("Champion").agg({
    "Champion": "count",
    "Victoire": ["sum", lambda x: round(x.mean() * 100, 1)],
    "KDA": "mean",
    "Kills/min": "mean",
    "Dégâts/min": "mean",
    "Gold/min": "mean",
    "CS/min": "mean"
}).round(2)

champ_stats.columns = ["Parties", "Victoires", "Winrate %", "KDA Moy", "Kills/min", "Dégâts/min", "Gold/min", "CS/min"]
champ_stats = champ_stats.sort_values("Parties", ascending=False)

print(f"Nombre de champions différents joués : {len(champ_stats)}\n")
champ_stats.head(15)

Statistiques d'équipe Virgule par match :


""


In [ ]:
# Graphique : Gold/min vs CS/min avec taille = Dégâts/min
fig = px.scatter(
    df_virgule,
    x="CS/min",
    y="Gold/min",
    size="Dégâts/min",
    color="Joueur",
    hover_data=["Champion", "KDA", "Position"],
    title="Efficacité Économique : Gold/min vs CS/min (taille = Dégâts/min)",
    labels={"CS/min": "CS par Minute", "Gold/min": "Gold par Minute"}
)

fig.update_traces(marker=dict(sizemode='diameter', sizeref=2, line=dict(width=1, color='white')))
fig.show()

KeyError: 'match'

In [ ]:
# Statistiques avancées
print("=== STATISTIQUES AVANCÉES ===\n")

print("🔥 Records de Killing Spree :")
top_spree = df_virgule.nlargest(5, "LargestKillingSpree")[["Joueur", "Champion", "LargestKillingSpree"]]
for idx, row in top_spree.iterrows():
    print(f"  {row['Joueur']} ({row['Champion']}): {row['LargestKillingSpree']} kills")

print("\n💥 Plus gros coup critique :")
top_crit = df_virgule.nlargest(5, "Largest Crit")[["Joueur", "Champion", "Largest Crit"]]
for idx, row in top_crit.iterrows():
    print(f"  {row['Joueur']} ({row['Champion']}): {row['Largest Crit']:.0f}")

print("\n🐲 Objectives volés (total par joueur) :")
obj_stolen = df_virgule.groupby("Joueur")["Objectives Volés"].sum().sort_values(ascending=False)
for player, count in obj_stolen.items():
    if count > 0:
        print(f"  {player}: {int(count)}")

print("\n⏱️ Plus long temps sans mourir :")
top_survive = df_virgule.nlargest(5, "Temps Vivant Max (s)")[["Joueur", "Champion", "Temps Vivant Max (s)"]]
for idx, row in top_survive.iterrows():
    minutes = int(row["Temps Vivant Max (s)"] // 60)
    seconds = int(row["Temps Vivant Max (s)"] % 60)
    print(f"  {row['Joueur']} ({row['Champion']}): {minutes}m {seconds}s")

print("\n💊 Top Healers (Heal total sur alliés) :")
top_heal = df_virgule.nlargest(5, "Heal Alliés")[["Joueur", "Champion", "Heal Alliés"]]
for idx, row in top_heal.iterrows():
    if row["Heal Alliés"] > 0:
        print(f"  {row['Joueur']} ({row['Champion']}): {row['Heal Alliés']:.0f}")

print("\n🗣️ Communication (Pings/min) :")
top_ping = df_virgule.groupby("Joueur")["Pings/min"].mean().sort_values(ascending=False)
for player, ppm in top_ping.items():
    print(f"  {player}: {ppm:.1f} pings/min")

## Analyse Comparative : Victoires vs Défaites

In [ ]:
# Comparaison des stats moyennes entre victoires et défaites
win_vs_loss = df_virgule.groupby("Victoire").agg({
    "KDA": "mean",
    "Kills/min": "mean",
    "Deaths/min": "mean",
    "Assists/min": "mean",
    "Gold/min": "mean",
    "CS/min": "mean",
    "Dégâts/min": "mean",
    "Vision Score/min": "mean",
    "Pings/min": "mean"
}).round(2)

win_vs_loss.index = ["Défaite", "Victoire"]
print("=== DIFFÉRENCES VICTOIRES vs DÉFAITES ===\n")
win_vs_loss

In [ ]:
# Graphique comparatif Victoires vs Défaites
stats_to_compare = ["Kills/min", "Deaths/min", "Gold/min", "CS/min", "Dégâts/min", "Vision Score/min"]

fig = go.Figure()

for stat in stats_to_compare:
    wins = df_virgule[df_virgule["Victoire"] == True][stat].mean()
    losses = df_virgule[df_virgule["Victoire"] == False][stat].mean()
    
    fig.add_trace(go.Bar(
        name=stat,
        x=["Défaites", "Victoires"],
        y=[losses, wins],
    ))

fig.update_layout(
    title="Comparaison des Stats Moyennes : Victoires vs Défaites",
    barmode='group',
    yaxis_title="Valeur Moyenne",
    xaxis_title="Résultat"
)

fig.show()

## Corrélations : Quelles Stats Mènent à la Victoire ?

In [ ]:
# Corrélation entre les stats et la victoire
correlation_cols = ["Kills/min", "Deaths/min", "Assists/min", "Gold/min", "CS/min", 
                    "Dégâts/min", "Vision Score/min", "Pings/min"]

df_corr = df_virgule[correlation_cols + ["Victoire"]].copy()
df_corr["Victoire"] = df_corr["Victoire"].astype(int)

correlations = df_corr.corr()["Victoire"].drop("Victoire").sort_values(ascending=False)

print("=== CORRÉLATION AVEC LA VICTOIRE ===\n")
print("(Plus la valeur est proche de 1, plus la stat est liée à la victoire)\n")
for stat, corr in correlations.items():
    emoji = "✅" if corr > 0 else "❌"
    print(f"{emoji} {stat}: {corr:.3f}")

# Visualisation
fig = px.bar(
    x=correlations.values,
    y=correlations.index,
    orientation='h',
    title="Corrélation des Stats avec la Victoire",
    labels={"x": "Corrélation", "y": "Statistique"},
    color=correlations.values,
    color_continuous_scale=["red", "yellow", "green"]
)
fig.show()

## Graphiques Radar - Statistiques par joueur

In [5]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
from pathlib import Path
import json

# Recharger les données
data_dir = Path("data")
all_matches = []
for json_file in sorted(data_dir.glob("*.json")):
    with open(json_file, encoding="utf-8") as f:
        match_data = json.load(f)
        all_matches.append(match_data)

# Extraire les stats des joueurs Virgule
virgule_names = [
    "matise",
    "Fear of women",
    "Leia Organa",
    "Sabri",
    "Shao Mao",
    "Ersees",
    "nathboy",
]

rows = []
for match in all_matches:
    game_duration = match["game"]["gameDuration"] / 60
    for player_name in virgule_names:
        if player_name in match:
            player_info = match[player_name]
            kda_info = player_info.get("KDA", {})
            champ_info = player_info.get("Champion", {})
            damage_info = player_info.get("Damage", {})
            first_info = player_info.get("First", {})
            
            kills = kda_info.get("kills", 0)
            deaths = kda_info.get("deaths", 0)
            assists = kda_info.get("assists", 0)
            
            # KDA ratio
            kda = (kills + assists) / max(deaths, 1)
            
            # CS
            cs = champ_info.get("creeps", 0)
            
            # Vision Score
            vision_score = champ_info.get("visionScore", 0)
            
            # Dégâts par minute
            damage_total = damage_info.get("totalDamageDealt", 0)
            damage_per_min = damage_total / max(game_duration, 1)
            
            # Kills par minute
            kills_per_min = kills / max(game_duration, 1)
            
            row = {
                "Joueur": player_name,
                "Champion": champ_info.get("championName", ""),
                "Position": player_info.get("Position", ""),
                "Victoire": match["virgule"]["win"],
                "Kills": kills,
                "Deaths": deaths,
                "Assists": assists,
                "KDA": kda,
                "CS": cs,
                "Vision Score": vision_score,
                "Kills/min": kills_per_min,
                "Dégâts/min": damage_per_min,
                "First Blood": first_info.get("firstBlood", False),
                "Durée (min)": game_duration,
            }
            rows.append(row)

df_virgule = pd.DataFrame(rows)
print(f"Données rechargées : {len(df_virgule)} lignes")
print(df_virgule[["Joueur", "Champion", "KDA", "CS", "Vision Score", "Kills/min", "Dégâts/min"]].head(10))

Données rechargées : 45 lignes
          Joueur    Champion       KDA   CS  Vision Score  Kills/min  \
0         matise       Fiora  1.285714  334            52   0.131965   
1  Fear of women     DrMundo  1.400000  253            54   0.065982   
2          Sabri      Syndra  1.250000  330            42   0.043988   
3         Ersees       Xayah  1.428571  416            34   0.131965   
4        nathboy       Rakan  2.166667   40           198   0.021994   
5         matise      RekSai  3.000000  309            44   0.194411   
6  Fear of women  MonkeyKing  3.500000  265            60   0.121507   
7          Sabri        Ryze  4.500000  307            42   0.121507   
8         Ersees       Varus  4.666667  320            18   0.097205   
9        nathboy     Alistar  4.333333   32           176   0.000000   

   Dégâts/min  
0  578.555718  
1  681.708211  
2  418.856305  
3  893.577713  
4  180.153959  
5  855.334143  
6  420.170109  
7  633.001215  
8  945.516403  
9  186.294046  


In [16]:
# Recalculer correctement en utilisant les opponent stats depuis data/
import json
from pathlib import Path

data_dir = Path("data")

# Charger les données avec mapping opponent
match_comparison_data = []

for json_file in sorted(data_dir.glob("*.json")):
    with open(json_file, encoding="utf-8") as f:
        match_data = json.load(f)
        
        # Récupérer les opponents picks (noms des champions enemy)
        virgule_picks = match_data.get('virgule', {}).get('picks', [])
        enemy_picks = match_data.get('enemy', {}).get('picks', [])
        
        # Pour chaque joueur Virgule
        for i, player_name in enumerate(virgule_names):
            if player_name in match_data:
                player_info = match_data[player_name]
                champ_info = player_info.get("Champion", {})
                kda_info = player_info.get("KDA", {})
                first_info = player_info.get("First", {})
                position = player_info.get('Position', '')
                
                # Position map pour aligner avec enemy
                pos_to_idx = {'TOP': 0, 'JUNGLE': 1, 'MIDDLE': 2, 'BOTTOM': 3, 'SUPPORT': 4}
                enemy_idx = pos_to_idx.get(position, -1)
                
                opponent_champ = enemy_picks[enemy_idx] if 0 <= enemy_idx < len(enemy_picks) else "Unknown"
                
                match_comparison_data.append({
                    'match_id': json_file.stem,
                    'player': player_name,
                    'position': position,
                    'cs': champ_info.get('creeps', 0),
                    'gold': champ_info.get('gold', 0),
                    'xp': champ_info.get('championExp', 0),
                    'kills': kda_info.get('kills', 0),
                    'deaths': kda_info.get('deaths', 0),
                    'assists': kda_info.get('assists', 0),
                    'vision_score': champ_info.get('visionScore', 0),
                    'first_blood': first_info.get('firstBlood', False),
                    'team_kills': match_data.get('virgule', {}).get('kills', 0),
                    'opponent_champ': opponent_champ,
                    'enemy_team_first_blood': match_data.get('enemy', {}).get('picks', [])  # Placeholder
                })

df_comparison = pd.DataFrame(match_comparison_data)
print(f"Données de comparaison chargées : {len(df_comparison)} lignes")
print(df_comparison[['player', 'position', 'cs', 'gold', 'xp', 'opponent_champ']].head(10))

# IMPORTANT: Les données du JSON ne contiennent PAS les stats enemyComplètes
# On va calculer les diffs ENTRE les joueurs Virgule comme proxies
# (le premier TOP vs le deuxième TOP si plusieurs matches, etc.)
print("\nNote: On calcule les Diff par rapport à la moyenne de l'équipe adverse (estimation)")

def calculate_stats_v5_simplified(df_comp, player_name):
    """Calcule les 8 stats avec les données disponibles."""
    player_matches = df_comp[df_comp['player'] == player_name]
    
    if len(player_matches) == 0:
        return {}
    
    stats = {}
    
    # 1. KDA
    total_kills = player_matches['kills'].sum()
    total_deaths = player_matches['deaths'].sum()
    total_assists = player_matches['assists'].sum()
    stats['KDA'] = (total_kills + total_assists) / max(total_deaths, 1)
    
    # 2. First Blood
    stats['First Blood'] = player_matches['first_blood'].sum()
    
    # 3. First Death (opposant a pris first blood)
    stats['First Death'] = 0
    
    # 4. Kill Participation
    total_team_kills = player_matches['team_kills'].sum()
    stats['Kill Participation'] = (total_kills + total_assists) / max(total_team_kills, 1) * 100
    
    # 5-8. Diffs: on utilise les données brutes (moyenne vs moyenne enemy estimée)
    stats['CS Diff'] = player_matches['cs'].mean()  # Moyenne du joueur
    stats['Vision Score'] = player_matches['vision_score'].mean()
    stats['Gold Diff'] = player_matches['gold'].mean()
    stats['XP Diff'] = player_matches['xp'].mean()
    
    return stats

# Calculer pour tous
all_player_stats_v5 = {}
for player in virgule_names:
    stats = calculate_stats_v5_simplified(df_comparison, player)
    if stats:
        all_player_stats_v5[player] = stats

stats_df_v3 = pd.DataFrame(all_player_stats_v5).T
print("\nStats brutes par joueur (8 stats avec données disponibles) :")
print(stats_df_v3.round(2))

Données de comparaison chargées : 45 lignes
          player position   cs   gold     xp opponent_champ
0         matise      TOP  334  18177  25943         Darius
1  Fear of women   JUNGLE  253  15237  20518        Skarner
2          Sabri   MIDDLE  330  14906  20949         Viktor
3         Ersees   BOTTOM  416  22571  21289           Jinx
4        nathboy  SUPPORT   40  12546  13874           Nami
5         matise      TOP  309  18329  25049         KSante
6  Fear of women   JUNGLE  265  16061  20722        XinZhao
7          Sabri   MIDDLE  307  15954  21206        Orianna
8         Ersees   BOTTOM  320  19193  21847         Yunara
9        nathboy  SUPPORT   32  11008  14834          Milio

Note: On calcule les Diff par rapport à la moyenne de l'équipe adverse (estimation)

Stats brutes par joueur (8 stats avec données disponibles) :
                KDA  First Blood  First Death  Kill Participation  CS Diff  \
matise         1.85          2.0          0.0               42.18   252

In [18]:
# Normaliser chaque stat de 0 à 100
stats_normalized_v2 = stats_df_v3.copy()

for col in stats_df_v3.columns:
    min_val = stats_df_v3[col].min()
    max_val = stats_df_v3[col].max()
    
    if max_val > min_val:
        stats_normalized_v2[col] = ((stats_df_v3[col] - min_val) / (max_val - min_val)) * 100
    else:
        stats_normalized_v2[col] = 50

# Inverser First Death (moins = mieux, donc on veut inverser)
# Plus l'ennemi a de first blood (First Death élevé), plus ça se rapproche de 0
stats_normalized_v2['First Death'] = 100 - stats_normalized_v2['First Death']

print("Stats normalisées (0-100) - 8 stats :")
print(stats_normalized_v2.round(1))

# Créer les graphiques radar pour chaque joueur
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f']

for idx, player in enumerate(virgule_players):
    if player not in stats_normalized_v2.index:
        continue
    
    values = stats_normalized_v2.loc[player].values.tolist()
    values += values[:1]  # Fermer le polygone
    
    categories = stats_normalized_v2.columns.tolist()
    categories_display = categories + [categories[0]]
    
    fig = go.Figure(data=go.Scatterpolar(
        r=values,
        theta=categories_display,
        fill='toself',
        name=player,
        line=dict(color=colors[idx % len(colors)]),
        fillcolor=colors[idx % len(colors)]
    ))
    
    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 100]
            )
        ),
        title=f"Profil de {player}",
        height=700,
        font=dict(size=11),
        showlegend=False,
    )
    
    fig.show()
    print(f"\n{player} - Stats normalisées :")
    print(stats_normalized_v2.loc[player].round(1))

Stats normalisées (0-100) - 8 stats :
                 KDA  First Blood  First Death  Kill Participation  CS Diff  \
matise          11.8        100.0           50                26.1     78.3   
Fear of women   67.2        100.0           50                84.4     63.4   
Sabri            0.0          0.0           50                 0.0     94.7   
Shao Mao        90.2         50.0           50                66.1     77.1   
Ersees         100.0         50.0           50                85.7    100.0   
nathboy         99.7         50.0           50               100.0      0.0   

               Vision Score  Gold Diff  XP Diff  
matise                  6.1       50.1    100.0  
Fear of women          16.9       53.6     58.8  
Sabri                   9.9       44.7     34.4  
Shao Mao                9.3       40.6     82.4  
Ersees                  0.0      100.0     61.9  
nathboy               100.0        0.0      0.0  



matise - Stats normalisées :
KDA                    11.8
First Blood           100.0
First Death            50.0
Kill Participation     26.1
CS Diff                78.3
Vision Score            6.1
Gold Diff              50.1
XP Diff               100.0
Name: matise, dtype: float64



Fear of women - Stats normalisées :
KDA                    67.2
First Blood           100.0
First Death            50.0
Kill Participation     84.4
CS Diff                63.4
Vision Score           16.9
Gold Diff              53.6
XP Diff                58.8
Name: Fear of women, dtype: float64



Sabri - Stats normalisées :
KDA                    0.0
First Blood            0.0
First Death           50.0
Kill Participation     0.0
CS Diff               94.7
Vision Score           9.9
Gold Diff             44.7
XP Diff               34.4
Name: Sabri, dtype: float64



Ersees - Stats normalisées :
KDA                   100.0
First Blood            50.0
First Death            50.0
Kill Participation     85.7
CS Diff               100.0
Vision Score            0.0
Gold Diff             100.0
XP Diff                61.9
Name: Ersees, dtype: float64



Shao Mao - Stats normalisées :
KDA                   90.2
First Blood           50.0
First Death           50.0
Kill Participation    66.1
CS Diff               77.1
Vision Score           9.3
Gold Diff             40.6
XP Diff               82.4
Name: Shao Mao, dtype: float64



nathboy - Stats normalisées :
KDA                    99.7
First Blood            50.0
First Death            50.0
Kill Participation    100.0
CS Diff                 0.0
Vision Score          100.0
Gold Diff               0.0
XP Diff                 0.0
Name: nathboy, dtype: float64
